Mounted at /content/drive


# Importing Libraries

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import numpy as np
import pandas as pd
import imageio
import random
import matplotlib.pyplot as plt
import keras
import tensorflow.keras as K
import tensorflow.keras.backend as Kback

from keras import layers
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers import Dropout
from keras.layers import Conv2DTranspose
from keras.layers import concatenate
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.callbacks import EarlyStopping
from keras.applications.resnet import ResNet50
from keras.applications import vgg16
from keras.applications import inception_v3
from keras.src.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications import DenseNet121, NASNetMobile, EfficientNetB0, Xception
from keras.layers import GlobalAveragePooling2D
from keras.applications.densenet import DenseNet121

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight


from collections import Counter
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import roc_auc_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Loadng


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Define the base path to your dataset
base_path = '/content/drive/My Drive/LUNGCANCER/IQ-OTH/The IQ-OTHNCCD lung cancer dataset'

# List of class names (A, B, C)
class_names =  ["Normal cases", "Malignant cases", "Bengin cases"]

# Initialize empty lists for images and labels
all_images = []
all_labels = []

# Load images and assign labels
for label, class_name in enumerate(class_names):
    class_path = os.path.join(base_path, class_name)
    image_files = os.listdir(class_path)

    for image_file in image_files:
        image_path = os.path.join(class_path, image_file)
        img = cv2.imread(image_path, cv2.IMREAD_COLOR)  # Read as GBR
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)


        # img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
        img = cv2.resize(img, (224, 224))  # Resize to desired dimensions
        img = img / 255.0  # Normalize pixel values (assuming 8-bit images)
        all_images.append(img)
        all_labels.append(label)


In [ ]:
data=all_images
labels=all_labels

In [ ]:
data, labels = shuffle(data, labels, random_state=42)


print('Data length:', len(data))
print('labels counts:', Counter(labels))

X = np.array(data).reshape(-1, 224, 224, 3)
y = np.array(labels)

Data length: 1097
labels counts: Counter({1: 561, 0: 416, 2: 120})


# Train Test Valid Split

In [ ]:
X_train, X_mix, y_train, y_mix = train_test_split(X, y, test_size=0.2, random_state=10)
X_test, X_valid, y_test, y_valid = train_test_split(X_mix, y_mix, test_size=0.2, random_state=10)
print('Train length:', len(X_train), X_train.shape)
print('Valid length:', len(X_valid), X_valid.shape)
print('Test length:', len(X_test), X_test)
train_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()
# X,y valid are used for evaluate matrix

print('Train length:', len(X_train), X_train.shape)
print('Valid length:', len(X_valid), X_valid.shape)
print('Test length:', len(X_test), X_test.shape)



# train_generator = train_datagen.flow(X_fold, y_fold, batch_size=8, shuffle=True)
# validation_generator = val_datagen.flow(X_valid, y_valid, batch_size=8, shuffle=True)

Train length: 877 (877, 224, 224, 3)
Valid length: 44 (44, 224, 224, 3)
Test length: 176 [[[[0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]
   ...
   [0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]]

  [[0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]
   ...
   [0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]]

  [[0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]
   ...
   [0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]]

  ...

  [[0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]
   ...
   [0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.149019

# RGB Split

In [ ]:
R_channel_train = []
R_channel_valid = []
R_channel_test=[]
for i in range(len(X_train)):
    R_channel_train.append(X_train[i][:,:,0])
R_channel_train = np.array(R_channel_train)

for i in range(len(X_valid)):
  R_channel_valid.append(X_valid[i][:,:,0])
R_channel_valid=np.array(R_channel_valid)

for i in range(len(X_test)):
  R_channel_test.append(X_test[i][:,:,0])
R_channel_test=np.array(R_channel_test)

In [ ]:
X_train.shape

(877, 224, 224, 3)

In [ ]:
R_channel_train.shape

(877, 224, 224)

In [ ]:
R_channel_valid.shape

(44, 224, 224)

In [ ]:
R_channel_test.shape

(176, 224, 224)

In [ ]:
G_channel_train = []
for i in range(len(X_train)):
    G_channel_train.append(X_train[i][:,:,1])
G_channel_train = np.array(G_channel_train)

G_channel_valid=[]
for i in range(len(X_valid)):
  G_channel_valid.append(X_valid[i][:,:,1])
G_channel_valid=np.array(G_channel_valid)

G_channel_test=[]
for i in range(len(X_test)):
  G_channel_test.append(X_test[i][:,:,1])
G_channel_test=np.array(G_channel_test)

In [ ]:
B_channel_train = []
for i in range(len(X_train)):
    B_channel_train.append(X_train[i][:,:,2])
B_channel_train = np.array(B_channel_train)

B_channel_valid=[]
for i in range(len(X_valid)):
  B_channel_valid.append(X_valid[i][:,:,2])
B_channel_valid=np.array(B_channel_valid)

B_channel_test=[]
for i in range(len(X_test)):
  B_channel_test.append(X_test[i][:,:,2])
B_channel_test=np.array(B_channel_test)

# Base Model + Features RGB + Features Actual + Combining Features


In [ ]:
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)

## Intermediate Model

In [ ]:
NUM_CLASSES = 3
input_layer = layers.Input(shape=(224, 224, 1))
densenet_model = Sequential()
densenet_model.add(input_layer)
densenet_model.add(Conv2D(3, (3, 3), padding='same'))
densenet_model.add(base_model)
densenet_model.add(layers.Flatten(name='flatten'))
# intermediate_features = densenet_model.layers[-1].output
# attention_weights = layers.Dense(1, activation='softmax')(intermediate_features)
# attention_weights = layers.Reshape((1, 1, -1))(attention_weights)
# densenet_model.add(layers.Dense(10, activation='relu'))
densenet_model.layers[0].trainable = False

In [ ]:
densenet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 3)       30        
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 flatten (Flatten)           (None, 62720)             0         
                                                                 
Total params: 2258014 (8.61 MB)
Trainable params: 2223872 (8.48 MB)
Non-trainable params: 34142 (133.37 KB)
_________________________________________________________________


### For Finding Individual Feature of RGB TrainTestValid

In [ ]:
# because rgb have 224X224x1 so we have to tweak model likewise

In [ ]:
from keras.models import Model
intermediate_model = Model(inputs=densenet_model.input, outputs=densenet_model.get_layer('flatten').output)
intermediate_output_r_channel = intermediate_model.predict(R_channel_train)
# intermediate_output_g_channel = intermediate_model.predict(G_channel_train)
# intermediate_output_b_channel = intermediate_model.predict(B_channel)

28/28 [==============================] - 49s 2s/step


In [ ]:
intermediate_output_r_channel.shape

(877, 62720)

In [ ]:
intermediate_output_g_channel = intermediate_model.predict(G_channel_train)

28/28 [==============================] - 50s 2s/step


In [ ]:
intermediate_output_g_channel.shape

(877, 62720)

In [ ]:

intermediate_output_b_channel = intermediate_model.predict(B_channel_train)

28/28 [==============================] - 51s 2s/step


In [ ]:
intermediate_output_b_channel.shape

(877, 62720)

In [ ]:
print(intermediate_model.input_shape)

(None, 224, 224, 1)


In [ ]:
intermediate_output_r_channel_valid = intermediate_model.predict(R_channel_valid)

2/2 [==============================] - 3s 712ms/step


In [ ]:
intermediate_output_g_channel_valid = intermediate_model.predict(G_channel_valid)

2/2 [==============================] - 2s 459ms/step


In [ ]:
intermediate_output_b_channel_valid = intermediate_model.predict(B_channel_valid)

2/2 [==============================] - 2s 434ms/step


In [ ]:
intermediate_output_r_channel_test = intermediate_model.predict(R_channel_test)

6/6 [==============================] - 10s 2s/step


In [ ]:
intermediate_output_g_channel_test = intermediate_model.predict(B_channel_test)

6/6 [==============================] - 10s 2s/step


In [ ]:
intermediate_output_b_channel_test = intermediate_model.predict(B_channel_test)

6/6 [==============================] - 8s 1s/step


## Intermediate Model for Actual Image

In [ ]:
densenet_model_main= Sequential()
densenet_model_main.add(base_model)
densenet_model_main.add(layers.Flatten(name='flatten'))
intermediate_features = densenet_model_main.layers[-1].output
attention_weights = layers.Dense(1, activation='softmax')(intermediate_features)
attention_weights = layers.Reshape((1, 1, -1))(attention_weights)
densenet_model_main.add(layers.Dense(10, activation='relu'))
densenet_model_main.layers[0].trainable = False

### For Finding Individual Feature of RGB as whole

In [ ]:
# because whole have 224X224x3 so we have to tweak model likewise

In [ ]:
intermediate_model_2= Model(inputs=densenet_model_main.input, outputs=densenet_model_main.get_layer('flatten').output)


In [ ]:
intermediate_output_main_train = intermediate_model_2.predict(X_train)

In [ ]:
intermediate_output_main_valid = intermediate_model_2.predict(X_valid)

In [ ]:
intermediate_output_main_test = intermediate_model_2.predict(X_test)

In [ ]:
intermediate_output_main_train.shape

In [ ]:
intermediate_output_main_valid.shape

In [ ]:
intermediate_output_main_test.shape

## Combined Features for Fitting

In [ ]:
combined_features = []
for i in range(len(intermediate_output_main_train)):
  combined_features.append(np.concatenate((intermediate_output_main_train[i], intermediate_output_r_channel[i], intermediate_output_g_channel[i], intermediate_output_b_channel[i]), axis=0))
combined_features = np.array(combined_features)
combined_features.shape

In [ ]:
combined_features_valid = []
for i in range(len(intermediate_output_main_valid)):
  combined_features_valid.append(np.concatenate((intermediate_output_main_valid[i], intermediate_output_r_channel_valid[i], intermediate_output_g_channel_valid[i], intermediate_output_b_channel_valid[i]), axis=0))
combined_features_valid = np.array(combined_features_valid)
combined_features_valid.shape

In [ ]:
combined_features_test = []
for i in range(len(intermediate_output_main_test)):
  combined_features_test.append(np.concatenate((intermediate_output_main_test[i], intermediate_output_r_channel_test[i], intermediate_output_g_channel_test[i], intermediate_output_b_channel_test[i]), axis=0))
combined_features_test = np.array(combined_features_test)
combined_features_test.shape

# Main Model

In [ ]:
input_layer = layers.Input(shape=(None,250880))
densenet_model_3= Sequential()
densenet_model_3.add(input_layer)
densenet_model_3.add(layers.Dense(256, activation='relu'))
densenet_model_3.add(layers.Dense(128, activation='relu'))
densenet_model_3.add(layers.Dense(64, activation='relu'))
densenet_model_3.add(layers.Dense(NUM_CLASSES, activation='softmax'))

In [ ]:
optimizer=keras.optimizers.Adam(learning_rate=0.001)
losses=keras.losses.SparseCategoricalCrossentropy()
metrics=['accuracy']
densenet_model_3.compile(optimizer=optimizer,loss=losses,metrics=metrics)

In [ ]:
densenet_model_3.summary()

In [ ]:
callback1 = EarlyStopping(monitor='val_accuracy', patience=10)
# callback2 = keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/Colab Notebooks/IQ_OTH_Dataset/Models_with_Attention_ChannelWise/DenseNet121_model.h5', monitor='val_accuracy', save_best_only=True)

history_densenetModel = densenet_model_3.fit(combined_features,y_train, epochs=2, batch_size=32, validation_data=(combined_features_valid,y_valid), callbacks=[callback1])

# Model Evaluation

In [ ]:
test_loss, test_accuracy = densenet_model_3.evaluate(combined_features_test,y_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

In [ ]:
y_pred = densenet_model_3.predict(combined_features_test, verbose=1)
y_pred_2 = np.argmax(y_pred, axis=1)

print(classification_report(y_valid, y_pred_2))

In [ ]:
cm= confusion_matrix(y_true=y_test, y_pred=y_pred_2)
target_names = ["Benign", "Malignant", "Normal"]
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=target_names)
disp.plot(cmap='Blues')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix')
plt.show

In [ ]:
history_dict = history_densenetModel.history

acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(acc_values) + 1)

line1 = plt.plot(epochs, val_acc_values, label = 'Validation/Test Accuracy')
line2 = plt.plot(epochs, acc_values, label = 'Training Accuracy')

plt.setp(line1)
plt.setp(line2)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)

line1 = plt.plot(epochs, val_loss_values, label = 'Validation/Test Loss')
line2 = plt.plot(epochs, loss_values, label = 'Training Loss')

plt.setp(line1)
plt.setp(line2)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()
plt.show

In [ ]:
#One vs One
roc_auc_score(y_valid, y_pred,multi_class='ovo')

In [ ]:
#One vs Rest
roc_auc_score(y_valid, y_pred,multi_class='ovr')